<a href="https://colab.research.google.com/github/BrooksA28/Repos/blob/main/TNASolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from sklearn.tree import DecisionTreeClassifier, export_text
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [13]:
df = pd.read_csv('drive/MyDrive/Radiation20250610_R.csv', encoding='unicode_escape')
df.head()

,Competence,CAG,Staff_Group,Service_Group,Department,Role,Position,Employee
0,Y,SBH CLINICAL SUPPORT DIVISION,Medical and Dental,ADULT CRITICAL CARE TRANSFER,PAN London Critical Care Service,Consultant,Locum Consultant - Adult Critical Care Retrieval,33293920
1,Y,SBH CLINICAL SUPPORT DIVISION,Medical and Dental,ADULT CRITICAL CARE TRANSFER,PAN London Critical Care Service,Consultant,Locum Consultant - Adult Critical Care Retrieval,32587479
2,Y,SBH CLINICAL SUPPORT DIVISION,Medical and Dental,ADULT CRITICAL CARE TRANSFER,PAN London Critical Care Service,Consultant,Locum Consultant - Adult Critical Care Retrieval,32642152
3,Y,SBH CLINICAL SUPPORT DIVISION,Medical and Dental,ADULT CRITICAL CARE TRANSFER,PAN London Critical Care Service,Consultant,Patient Experience & Compliance Lead,32256161
4,Y,SBH CLINICAL SUPPORT DIVISION,Medical and Dental,ADULT CRITICAL CARE TRANSFER,PAN London Critical Care Service,Consultant,Locum Consultant - Adult Critical Care Retrieval,32582966


Decision Tree Attempt

Conclusion: Probably not appropriate as there are too many distinct features within the separate columns... Position in particular...

[Decision Tree Link](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

In [20]:
X = pd.get_dummies(df[['CAG', 'Staff_Group', 'Service_Group', 'Department', 'Role', 'Position']])
Y = df['Competence'].apply(lambda x: 1 if x == 'Y' else 0)

In [16]:
clf = DecisionTreeClassifier(max_depth=7)
clf.fit(X,Y)

print(export_text(clf, feature_names=list(X.columns)))

|--- Staff_Group_Nursing and Midwifery Registered <= 0.50
|   |--- Staff_Group_Additional Clinical Services <= 0.50
|   |   |--- Role_Radiographer - Diagnostic <= 0.50
|   |   |   |--- Service_Group_RLH IMAGING <= 0.50
|   |   |   |   |--- Service_Group_WXH EMERGENCY CARE & ACUTE MEDICINE <= 0.50
|   |   |   |   |   |--- Service_Group_RLH ED & HEMS <= 0.50
|   |   |   |   |   |   |--- Role_Radiographer - Therapeutic <= 0.50
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |--- Role_Radiographer - Therapeutic >  0.50
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- Service_Group_RLH ED & HEMS >  0.50
|   |   |   |   |   |   |--- Staff_Group_Medical and Dental <= 0.50
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |--- Staff_Group_Medical and Dental >  0.50
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |--- Service_Group_WXH EMERGENCY CARE & ACUTE MEDICINE >  0.50
|   |   |   |   |   |--- Staff_Group_Medical and Dental <= 

Association Rule Mining

In [21]:
df2 = df[df['Competence'] == 'Y']

X['Competence=Y'] = True

In [22]:
frequent_items = apriori(X, min_support=0.01, use_colnames=True)

rules = association_rules(frequent_items, metric='confidence', min_threshold=0.7)
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

                                            antecedents  \
0                              (CAG_CORPORATE SERVICES)   
1                     (Service_Group_EDUCATION ACADEMY)   
2                              (CAG_CORPORATE SERVICES)   
3                          (CAG_ESTATES AND FACILITIES)   
4                   (Staff_Group_Estates and Ancillary)   
...                                                 ...   
2373  (Service_Group_SOFT FM, Department_RLH - Clean...   
2374              (Service_Group_SOFT FM, Role_Cleaner)   
2375          (Department_RLH - Cleaning, Role_Cleaner)   
2376                        (Department_RLH - Cleaning)   
2377                                     (Role_Cleaner)   

                                            consequents   support  confidence  \
0             (Staff_Group_Administrative and Clerical)  0.054599    0.843365   
1                              (CAG_CORPORATE SERVICES)  0.010362    1.000000   
2                                        (Compet

/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/association_rules.py:186: RuntimeWarning: invalid value encountered in divide
  cert_metric = np.where(certainty_denom == 0, 0, certainty_num / certainty_denom)
